In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle
import camutils as cam

In [2]:
resultfile = 'calibration.pickle'
fid = open(resultfile, "rb")
a = pickle.load(fid)
fid.close()
fx, fy, cx, cy = a['fx'], a['fy'], a['cx'], a['cy']

In [3]:
favg = (fx+fy)/2

# create Camera objects representing the left and right cameras
# use the known intrinsic parameters you loaded in.
camL = cam.Camera(f=favg,c=np.array([[cx,cy]]).T,t=np.array([[0,0,0]]).T, R=cam.makerotation(0,0,0))
camR = cam.Camera(f=favg,c=np.array([[cx,cy]]).T,t=np.array([[0,0,0]]).T, R=cam.makerotation(0,0,0))

# load in the left and right images and find the coordinates of
# the chessboard corners using OpenCV
imgL = plt.imread('calib_jpg_u/frame_C0_01.jpg')
ret, cornersL = cv2.findChessboardCorners(imgL, (8,6), None)
pts2L = cornersL.squeeze().T

imgR = plt.imread('calib_jpg_u/frame_C1_01.jpg')
ret, cornersR = cv2.findChessboardCorners(imgR, (8,6), None)
pts2R = cornersR.squeeze().T

# generate the known 3D point coordinates of points on the checkerboard in cm
pts3 = np.zeros((3,6*8))
yy,xx = np.meshgrid(np.arange(8),np.arange(6))
pts3[0,:] = 2.8*xx.reshape(1,-1)
pts3[1,:] = 2.8*yy.reshape(1,-1)


# Now use your calibratePose function to get the extrinsic parameters
# for the two images. You may need to experiment with the initialization
# in order to get a good result
params_init = np.array([0,0,0,0,0,-2]) 

camL = cam.calibratePose(pts3,pts2L,camL,params_init)
camR = cam.calibratePose(pts3,pts2R,camR,params_init)

In [4]:
fid = open("cameras.pickle", "wb")
v = (camL, camR)
pickle.dump(v, fid)
fid.close()